In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve,least_squares,root

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
p = RollRig.base_params(1)

In [3]:
sm = mf.SymbolicModel.from_file('RollRigModel-Fixed.py')
sm.ExtForces = ef.CompositeForce([sm.ExtForces,ef.CustomForce(None)])
sm = sm.subs({p.alpha_r.name:0})

In [4]:
sm.ExtForces.Q

<bound method CompositeForce.Q of <ModelFramework.ExternalForces.CompositeForce.CompositeForce object at 0x7fd948190590>>

In [51]:
p.c.value = 0.0677
p.s.value = 1
p.sigma.value = 0.272
p.m_w.value = 0.22


p.eta_0.value = 0
p.eta_1.value = 0

p.alpha_r.value = 0
p.c_d_max.value = 0

p.a_0.value = 2*np.pi
p.a_1.value = 2*np.pi


p.w_g.value = 0
p.V.value = 22.5

#p.s.value = 0.72
#p.sigma.value = 0
#p.m_f.value = 0
#p.m_l.value = 0


p.Lambda.value = 0

In [52]:
nm = mf.NumericModel.from_SymbolicModel(p,sm)





In [53]:
def GenRunData(model,p,ic,end_time,xNames=None,additional_cols = {},sample_freq=100):
    
    if xNames==None:
        xNames = [f'x{i}' for i in range(p.qs*2)]
    
    # create objective function and solve
    def objectiveDeriv(t,y):
        return nm.deriv(t,y,p.GetNumericTuple(y,t))
    yData = solve_ivp(objectiveDeriv,(0,end_time),ic,max_step=1/sample_freq)
    
    t = np.linspace(0,end_time,(end_time*100)+1)
   
    #interpolate to reduce t and y points
    int_func = interp1d(yData.t,yData.y)  
    yi = int_func(t)
    
    #generate list
    q_data = [{names[i]:v for i,v in enumerate(row)}for row in yi.T]    
    params = {k.name:v for k,v in p.GetSubs(0,None).items()}


    data = [{'t':_t,'Locked':True,**q_data[i],**params,**additional_cols} for i,_t in enumerate(t)]
    return data
    

In [54]:
p.I_xx_w.value

In [69]:
names = ['Roll','Roll Rate','Left FWT Angle','Left FWT Velocity','Right FWT Angle','Right FWT Velocity']

res = []

for mode in ['Locked','Removed']:
    if mode =='Locked':
        p.m_w.value = 0.884
        p.I_xx_w.value = 0.03978611
        p.eta_0.value = 0.00047
        p.eta_1.value = -0.00238
        p.a_1.value = 2*np.pi
    else:
        p.m_w.value = 0.735
        p.I_xx_w.value = 0.014820479
        p.eta_0.value = 0.0020145
        p.eta_1.value = -0.002775
        p.a_1.value = 0
    for v in np.linspace(15,30,4):
        print(p.I_xx_w.value)
        p.V.value = v
        
        ext_f = nm.ExtForces.force_funcs[0]
        torque_period = 0.1     
        aas = [7,14,21]
        offset = [-0.13,-0.10,0.05]
        for i,tm in enumerate([1.57,2.31,2.75]):
            p.beta.value = (tm)*8.4e-4
            torque_max = p.beta.value*p.V.value**2+offset[i]

            def torque(tup,x,t):
                if t<2:
                    return 0
                elif t<torque_period/2+2:
                    return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
                else:
                    return torque_max
            def Aileron_drag(tup,x,t):
                return -np.abs(x[1])*x[1]*0.0

            def custom_force(tup,x,t):
                return np.array([[torque(tup,x,t)+Aileron_drag(tup,x,t)]])

            nm.ExtForces.force_funcs = [ext_f , custom_force]

            res+=GenRunData(nm,p,[0,0],10,names[:2],{'Mode':mode,'AileronAngle':aas[i],**{n:0 for n in names[2:]}},100)
df = pd.DataFrame(res)
df.to_pickle('FixedData.pkl')

0.03978611
0.03978611
0.03978611
0.03978611
0.014820479
0.014820479
0.014820479
0.014820479


In [70]:
df['Roll Rate [Deg]']=np.rad2deg(df['Roll Rate'])
df[df['t']>4].groupby(['Mode','V','AileronAngle'])['Roll Rate [Deg]'].mean()

Mode     V     AileronAngle
Locked   15.0  7                34.500135
               14               70.895164
               21              120.595817
         20.0  7                62.911253
               14              107.460163
               21              154.504014
         25.0  7                88.699024
               14              141.515416
               21              189.735201
         30.0  7               111.843647
               14              174.241198
               21              225.460679
Removed  15.0  7                75.547008
               14              152.512105
               21              259.548569
         20.0  7               136.295791
               14              231.341509
               21              332.956399
         25.0  7               189.591177
               14              304.446663
               21              408.697451
         30.0  7               240.886357
               14              375.158412
      

In [71]:
df.to_csv(r'/Users/fintan/Git/FlexiFWT_DAQ/Analysis/RollingRigV3/ModelComparison/FixedData.csv', index = False)